In [1]:
from massspecgym.data.datasets import MSnDataset, MSnRetrievalDataset, RetrievalDataset
import torch

In [2]:
file_mgf = "/Users/macbook/CODE/Majer:MassSpecGym/data/MSn/20241211_msn_library_pos_all_lib_MSn.mgf"
file_json = "/Users/macbook/CODE/Majer:MassSpecGym/data/Retrieval/MassSpecGym_retrieval_candidates_mass.json"
split_file = "/Users/macbook/CODE/Majer:MassSpecGym/data/MSn/20241211_split.tsv"

In [3]:
from massspecgym.data.transforms import MolFingerprinter, SpecTokenizer
from massspecgym.data import RetrievalDataset, MassSpecDataModule

In [4]:
pth_massspecgym_original = "/Users/macbook/CODE/Majer:MassSpecGym/data/MassSpecGym/MassSpecGym.tsv"

In [5]:
# Init hyperparameters
n_peaks = 60
fp_size = 4096
batch_size = 12

# Load dataset
dataset_original = RetrievalDataset(
    pth=pth_massspecgym_original,
    candidates_pth=file_json,
    spec_transform=SpecTokenizer(n_peaks=n_peaks),
    mol_transform=MolFingerprinter(fp_size=fp_size),
)


In [6]:
# Init data module
data_module_original = MassSpecDataModule(
    dataset=dataset_original,
    batch_size=batch_size,
    num_workers=0,
)

In [7]:
data_module_original.prepare_data()
data_module_original.setup()

train_loader_original = data_module_original.train_dataloader()

Train dataset size: 194119
Val dataset size: 19429


In [8]:
tmp_original = []
for batch in train_loader_original:
    print(batch)
    tmp_original = batch
    break

I am not here
{'spec': tensor([[[3.2114e+02, 1.1000e+00],
         [4.1038e+01, 1.0710e-02],
         [4.2034e+01, 1.5940e-02],
         ...,
         [2.3407e+02, 6.6300e-03],
         [2.3608e+02, 1.5070e-02],
         [2.6206e+02, 1.7320e-02]],

        [[7.8565e+02, 1.1000e+00],
         [9.8984e+01, 8.0546e-03],
         [1.0411e+02, 7.5243e-03],
         ...,
         [1.8436e+02, 1.4384e-03],
         [2.6225e+02, 2.7558e-03],
         [5.8458e+02, 1.9532e-03]],

        [[1.5612e+02, 1.1000e+00],
         [4.4049e+01, 8.4590e-02],
         [4.4053e+01, 2.4700e-03],
         ...,
         [0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00]],

        ...,

        [[2.6714e+02, 1.1000e+00],
         [2.4516e+02, 1.4000e-02],
         [2.5109e+02, 7.0000e-03],
         ...,
         [0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00]],

        [[5.0610e+02, 1.1000e+00],
         [3.2511e+02, 1.

In [9]:
# Check the keys in the batch
print(tmp_original.keys())

# Specifically check if 'batch_ptr' is present
if 'batch_ptr' in tmp_original:
    print("batch_ptr is present:", tmp_original['batch_ptr'])
else:
    print("batch_ptr is missing")

dict_keys(['spec', 'mol', 'precursor_mz', 'adduct', 'mol_freq', 'identifier', 'smiles', 'candidates', 'labels', 'batch_ptr', 'candidates_smiles'])
batch_ptr is present: tensor([256, 214, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256])


In [10]:
tmp_original.keys()

dict_keys(['spec', 'mol', 'precursor_mz', 'adduct', 'mol_freq', 'identifier', 'smiles', 'candidates', 'labels', 'batch_ptr', 'candidates_smiles'])

In [11]:
tmp_original['spec'].shape, tmp_original['mol'].shape, 

(torch.Size([12, 61, 2]), torch.Size([12, 4096]))

In [12]:
tmp_original['precursor_mz']

tensor([321.1380, 785.6540, 156.1244, 241.0963, 703.3660, 248.0706, 502.1773,
        209.1280, 579.1500, 267.1400, 506.1000, 338.1751])

In [13]:
tmp_original['adduct'], 

(['[M+H]+',
  '[M+H]+',
  '[M+H]+',
  '[M+H]+',
  '[M+Na]+',
  '[M+H]+',
  '[M+H]+',
  '[M+H]+',
  '[M+H]+',
  '[M+Na]+',
  '[M+Na]+',
  '[M+H]+'],)

In [14]:
tmp_original['mol_freq']

tensor([ 3., 26.,  3., 48.,  2.,  6.,  4., 80., 31.,  7., 27., 36.])

In [15]:
tmp_original['labels']

tensor([ True, False, False,  ..., False, False, False])

In [16]:
sum(tmp_original['labels'])

tensor(12)

In [17]:
torch.nonzero(tmp_original['labels'], as_tuple=False).squeeze()

tensor([   0,  256,  470,  726,  982, 1238, 1494, 1750, 2006, 2262, 2518, 2774])

In [18]:
len(tmp_original['labels'])

3030

In [19]:
tmp_original['batch_ptr']

tensor([256, 214, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256])

In [20]:
sum(tmp_original['batch_ptr'])

tensor(3030)

In [21]:
len(tmp_original['candidates'])

3030

In [22]:
tmp_original['candidates'][5].shape

torch.Size([4096])

In [23]:
len(tmp_original['candidates_smiles'])

3030

# MSnRetrieval

In [24]:
from massspecgym.featurize import SpectrumFeaturizer

In [25]:
config = {
    'features': ['collision_energy', 'ionmode', 'adduct', 'spectrum_stats', 'atom_counts', 'value', "retention_time", 'ion_source', 'binned_peaks'],
    'feature_attributes': {
        'atom_counts': {
            'top_n_atoms': 12,
            'include_other': True,
        },
    },
}

In [26]:
featurizer = SpectrumFeaturizer(config, mode='torch')

In [27]:
file_json = "/Users/macbook/CODE/Majer:MassSpecGym/data/candidates_generation/MassSpecGymMSn_retrieval_candidates_formula.json"

In [28]:
# file_mgf = "/Users/macbook/CODE/Majer:MassSpecGym/data/MSn/min_sample_trees.mgf"


In [29]:
# Init hyperparameters
n_peaks = 60
fp_size = 4096
batch_size = 12

# Load dataset
dataset_msn = MSnRetrievalDataset(
    pth=file_mgf,
    candidates_pth=file_json,
    featurizer=featurizer,
    mol_transform=MolFingerprinter(fp_size=fp_size),
    max_allowed_deviation=0.005
)


Total valid indices: 16476
MSnRetrievalDataset length: 16476
Precomputing candidate-side transformations using multiprocessing.Pool with chunking and writing to HDF5...
Using 8 worker processes for precomputation.


Processing chunks: 100%|██████████| 33/33 [15:53<00:00, 28.90s/it]  


Precomputation complete. Cache saved to /Users/macbook/CODE/Majer:MassSpecGym/data/candidates_generation/MassSpecGymMSn_retrieval_candidates_formula.h5


In [31]:
# Init data module
data_module_msn = MassSpecDataModule(
    dataset=dataset_msn,
    batch_size=batch_size,
    split_pth=split_file,
    num_workers=0,
)

In [32]:
data_module_msn.prepare_data()
data_module_msn.setup()

train_loader_msn = data_module_msn.train_dataloader()

Train dataset size: 12536
Val dataset size: 1952


In [33]:
tmp_msn = []
for batch in train_loader_msn:
    print(batch)
    tmp_msn = batch
    break

__getitem__ timing breakdown for idx 6384:
  Map valid index: 0.02 ms
  Parent __getitem__: 501.26 ms
  Retrieve SMILES: 0.01 ms
  Set SMILES: 0.00 ms
  Candidate SMILES lookup: 0.03 ms
  HDF5 group lookup: 0.83 ms
  Mol retrieval from HDF5: 0.40 ms
  Mol tensor conversion: 0.37 ms
  Candidates retrieval from HDF5: 12.47 ms
  Candidates tensor conversion: 0.02 ms
  Labels retrieval from HDF5: 0.24 ms
  Labels conversion: 0.00 ms
  Total __getitem__: 515.65 ms
[PROFILE] __getitem__ took 0.515795 seconds
__getitem__ timing breakdown for idx 14760:
  Map valid index: 0.01 ms
  Parent __getitem__: 29.54 ms
  Retrieve SMILES: 0.00 ms
  Set SMILES: 0.00 ms
  Candidate SMILES lookup: 0.03 ms
  HDF5 group lookup: 0.46 ms
  Mol retrieval from HDF5: 0.48 ms
  Mol tensor conversion: 0.03 ms
  Candidates retrieval from HDF5: 8.17 ms
  Candidates tensor conversion: 0.01 ms
  Labels retrieval from HDF5: 0.16 ms
  Labels conversion: 0.00 ms
  Total __getitem__: 38.89 ms
[PROFILE] __getitem__ took 0.0

In [34]:
# Check the keys in the batch
print(tmp_msn.keys())

# Specifically check if 'batch_ptr' is present
if 'batch_ptr' in tmp_msn:
    print("batch_ptr is present:", tmp_msn['batch_ptr'])
else:
    print("batch_ptr is missing")

dict_keys(['spec', 'mol', 'precursor_mz', 'adduct', 'identifier', 'mol_freq', 'smiles', 'candidates', 'labels', 'batch_ptr', 'candidates_smiles'])
batch_ptr is present: tensor([256, 149, 256, 256, 256, 256, 256, 101, 256, 256, 256,   5],
       dtype=torch.int32)


In [35]:
def print_key_types(obj, obj_name):
    print(f"Data types in {obj_name}:")
    for key, value in obj.items():
        print(f"  {key}: {type(value)}")

# Print data types for both objects
print_key_types(tmp_original, "tmp_original")
print_key_types(tmp_msn, "tmp_msn")

Data types in tmp_original:
  spec: <class 'torch.Tensor'>
  mol: <class 'torch.Tensor'>
  precursor_mz: <class 'torch.Tensor'>
  adduct: <class 'list'>
  mol_freq: <class 'torch.Tensor'>
  identifier: <class 'list'>
  smiles: <class 'list'>
  candidates: <class 'torch.Tensor'>
  labels: <class 'torch.Tensor'>
  batch_ptr: <class 'torch.Tensor'>
  candidates_smiles: <class 'list'>
Data types in tmp_msn:
  spec: <class 'abc.DataBatch'>
  mol: <class 'torch.Tensor'>
  precursor_mz: <class 'torch.Tensor'>
  adduct: <class 'list'>
  identifier: <class 'list'>
  mol_freq: <class 'torch.Tensor'>
  smiles: <class 'list'>
  candidates: <class 'torch.Tensor'>
  labels: <class 'torch.Tensor'>
  batch_ptr: <class 'torch.Tensor'>
  candidates_smiles: <class 'list'>


#### Shold be dict_keys(['spec', 'mol', 'precursor_mz', 'adduct', 'mol_freq', 'identifier', 'smiles', 'candidates', 'labels', 'batch_ptr', 'candidates_smiles'])

In [36]:
tmp_msn.keys()

dict_keys(['spec', 'mol', 'precursor_mz', 'adduct', 'identifier', 'mol_freq', 'smiles', 'candidates', 'labels', 'batch_ptr', 'candidates_smiles'])

In [37]:
tmp_msn['spec'] 

DataBatch(x=[174, 1039], edge_index=[2, 324], batch=[174], ptr=[13])

In [38]:
tmp_msn['precursor_mz']

tensor([413.1642, 304.2999, 382.2026, 337.2023, 393.0737, 402.1482, 397.1329,
        358.1777, 358.2489, 385.2373, 469.1857, 695.3424])

In [39]:
tmp_msn['mol_freq']

tensor([1., 2., 1., 1., 1., 1., 1., 2., 1., 1., 2., 1.])

In [40]:
tmp_msn['identifier']

['0039149_0000000',
 '0090271_0000000',
 '0062970_0000000',
 '0035188_0000000',
 '0025403_0000000',
 '0047232_0000000',
 '0062236_0000000',
 '0073480_0000000',
 '0051923_0000000',
 '0072588_0000000',
 '0017855_0000000',
 '0096736_0000000']

In [41]:
tmp_msn['adduct']

['[M+H]+',
 '[M]+',
 '[M+H]+',
 '[M+H]+',
 '[M+H]+',
 '[M+H]+',
 '[M+H]+',
 '[M+H]+',
 '[M+H]+',
 '[M+H]+',
 '[M+H-H2O]+',
 '[M+H]+']

In [42]:
tmp_msn['mol'].shape

torch.Size([12, 4096])

In [43]:
tmp_msn['smiles']

['CN(CC(=O)NC1CCCC1)S(=O)(=O)C2=CC3=C(C=C2)N=C(N3)C4=CC=CC=C4',
 'CCCCCCCCCCCCCCCC[N+]1=CC=CC=C1',
 'CN1CCN(CC1)C2=NN=C(C(=C2)C3=CC=NC=C3)C4=CC5=CC=CC=C5C=C4',
 'CC(C)CN1C2=C(C=CC=N2)N=C1C3=CC(=CC=C3)C(=O)NC(C)C',
 'CCCCN1C(=O)C2=C(C=C(S2)C3=CC=C(C=C3)F)N=C1SCC(=O)O',
 'CCOC1=C(C=C(C=C1)NC(=O)C)S(=O)(=O)NCCC2=CNC3=CC=CC=C32',
 'COC1=C(C=CC(=C1)N2CCOCC2)NC(=O)C3=CSC(=N3)C4=CN=CC=C4',
 'C[C@H](C1=CC=CC2=CC=CC=C21)NCCCC3=CC(=CC=C3)C(F)(F)F',
 'C1CCC(CC1)NC(=O)N[C@@H](CC2=CC=CC=C2)C(=O)N3CCCCC3',
 'C[C@]12CC[C@@H](C[C@H]1CC[C@@H]3[C@@H]2CC[C@]4([C@]35[C@H](O5)C[C@@H]4C6=COC(=O)C=C6)C)O',
 'CC/C=C/C=C/C1(C(=O)C(=C(O1)OC)C(=O)C2C(C(=O)C3=CC(=C(C(=C3C2(C)O)O)O)OC)C)C',
 'C1[C@H](CN([C@@H]1C(=O)N[C@@H](CCCN=C(N)N)C(=O)NCC(=O)N[C@@H](CC2=CNC3=CC=CC=C32)C(=O)N)C(=O)[C@H](CC4=CC=C(C=C4)F)N)O']

In [44]:
tmp_msn['labels']

tensor([ True, False, False,  ..., False, False, False])

In [45]:
sum(tmp_msn['labels'])

tensor(12)

In [46]:
torch.nonzero(tmp_msn['labels'], as_tuple=False).squeeze()

tensor([   0,  256,  405,  661,  917, 1173, 1429, 1685, 1786, 2042, 2298, 2554])

In [47]:
len(tmp_msn['labels'])

2559

In [48]:
tmp_msn['batch_ptr']

tensor([256, 149, 256, 256, 256, 256, 256, 101, 256, 256, 256,   5],
       dtype=torch.int32)

In [49]:
sum(tmp_msn['batch_ptr'])

tensor(2559, dtype=torch.int32)

In [50]:
len(tmp_msn['candidates'])

2559

In [51]:
tmp_msn['candidates'][5].shape

torch.Size([4096])

### Compare Shapes

In [52]:
tmp_msn.keys() == tmp_original.keys()

True

In [53]:
tmp_msn['batch_ptr'].shape == tmp_original['batch_ptr'].shape

True

In [54]:
tmp_msn['batch_ptr'].dtype == tmp_original['batch_ptr'].dtype

False

In [55]:
tmp_original['batch_ptr'].dtype

torch.int64

In [56]:
tmp_msn['batch_ptr'].dtype

torch.int32

In [57]:
sum(tmp_msn['batch_ptr']), len(tmp_msn['candidates_smiles'])

(tensor(2559, dtype=torch.int32), 2559)

In [58]:
sum(tmp_original['batch_ptr']), len(tmp_original['candidates_smiles'])

(tensor(3030), 3030)

In [59]:
tmp_msn['labels'].dtype == tmp_original['labels'].dtype

True

In [60]:
tmp_msn['candidates'].dtype == tmp_original['candidates'].dtype

True

In [61]:
tmp_msn['candidates'][0].shape == tmp_original['candidates'][0].shape

True

In [62]:
tmp_msn['candidates'][5].shape == tmp_original['candidates'][5].shape

True

In [63]:
type(tmp_msn['smiles']) == type(tmp_original['smiles'])

True

In [64]:
type(tmp_msn['smiles'][0]) == type(tmp_original['smiles'][0])

True

In [65]:
len(tmp_msn['smiles']) == len(tmp_original['smiles'])

True

In [66]:
type(tmp_msn['candidates_smiles']) == type(tmp_original['candidates_smiles'])

True

In [67]:
type(tmp_msn['candidates_smiles'][0]) == type(tmp_original['candidates_smiles'][0])

True

In [68]:
len(tmp_msn['candidates_smiles'])

2559